# libraries

In [1]:
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm 
from datetime import datetime

# First Part (create the data base)

In [2]:
def CreateDataBase(database, user, password, host, port, newBase):
    
    '''
   function that creates the database

    database str : database name
    user str : user name
    password str : paswowrd of database
    host str: host of database
    port str: number of port
    newBase str: dabase to be create
    :return: null
   '''

    sqlQuery = 'CREATE database ' + newBase
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))
    conn = engine.connect()
    conn.execute("commit")

    try:
        conn.execute(sqlQuery)
        print("Database " + newBase +" created successfully........")
    except:
        print("Database " + newBase +" created unsuccessfully........")        
    
    conn.close()


    engine.dispose()


In [3]:
CreateDataBase(database="postgres", user='postgres', password='20906540', host='127.0.0.1', port= '5432', newBase = "factory" )

Database factory created successfully........


# Second Part (Creat a table from a DataFrame)

In [4]:
def createTable(data, table, database, user, password, host, port):
    
    '''
   function that creates the table
   
    data pandas.DataFrame: data to be insert
    table str: name of table
    database str : database name
    user str : user name
    password str : paswowrd of database
    host str: host of database
    port str: number of port
    newBase str: dabase to be create
    :return: null
   '''
    
    
    data.location = data.location.apply(lambda x: str(x).replace("'", "’"))
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))

    k = []
    for n in range(len(data.dtypes)):
        if data.dtypes[n] == 'object':
            k.append(data.dtypes.index[n] + " TEXT")
        if data.dtypes[n] == 'int64' or data.dtypes[n] == 'float64':
            k.append(data.dtypes.index[n] + " FLOAT")


    k1 = list(data.dtypes.index)

    sql = 'CREATE TABLE '+table+'(' + ", ".join(k) + ", UNIQUE (" + ", ".join(k1)+"))" 
    data = data.fillna("NoValue")
    data = data.set_index(data.columns[0])
    
    try:
        engine.execute(sql)
    except:
        print("table" + table +" created unsuccessfully........")

    data = data.reset_index()



    for m in tqdm(range(len(data))):
        k1 = list(data.loc[m].fillna("NoValue").index)
        k1 = [str(a) for a in k1]
        k1 = ", ".join(k1)


        k = data.loc[m].fillna("NoValue").to_list()
        for i in range(len(k)):
            if str(type(k[i])) == "<class 'str'>":
                k[i] = "'" + k[i] + "'"
        k = [str(a) for a in k]
        k = [a.replace(",", "") for a in k]
        k = ", ".join(k)

        try:
            sql = '''INSERT INTO public.'''+ table +'''('''+k1+ ") VALUES ("+k+ ") ON CONFLICT DO NOTHING"
            engine.execute(sql)
        except:
            print("Fila " + str(m) +" no insertada")


In [5]:
data = pd.read_csv("table_1.csv")

In [6]:
createTable(data = data, table = "tabla", database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432')

 76%|██████████████████████████████████████████████████████████▍                  | 1633/2154 [00:03<00:01, 414.97it/s]

Fila 1580 no insertada


100%|█████████████████████████████████████████████████████████████████████████████| 2154/2154 [00:05<00:00, 417.66it/s]


# Third Part (Crear View)

In [7]:
def Views(database, user, password, host, port, sqlQuery):
    
    '''
   function that creates a view
   
    database str : database name
    user str : user name
    password str : paswowrd of database
    host str: host of database
    port str: number of port
    sqlQuery str: Query to be execute 
    :return: null
   '''
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))


    sql = sqlQuery
    try:
        engine.execute(sql)
    except:
        print("View " + " created unsuccessfully........") 
            
    engine.dispose()

In [8]:
sqlQuery ='''CREATE VIEW Vista1 AS
SELECT  keyword, id , date,
AVG(positive) as avg_positive,
AVG(neutral) as avg_neutral,
AVG(negative) as avg_negative
FROM    tabla
GROUP BY 
keyword, id , date
'''

Views(database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432', sqlQuery = sqlQuery )

In [9]:
sqlQuery ='''CREATE VIEW Vista2 AS
SELECT  date,keyword, 
sum(CASE WHEN positive>negative AND positive>neutral THEN 1 ELSE 0 END) AS count_positive,
sum(CASE WHEN negative>positive AND negative>neutral THEN 1 ELSE 0 END) AS count_negative,
sum(CASE WHEN neutral>positive AND neutral>negative THEN 1 ELSE 0 END) AS count_neutral
FROM  tabla
GROUP BY
date,keyword
'''

Views(database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432', sqlQuery = sqlQuery )


# Four Part (Insert without Duplicated)

In [10]:
def InsertInTable(data, table, database, user, password, host, port):
    
    '''
   function that insert from DataFrame
   
    data pandas.DataFrame:: data to be insert
    database str : database name
    user str : user name
    password str : paswowrd of database
    host str: host of database
    port str: number of port
    :return: null
   '''
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))

    data.location = data.location.apply(lambda x: str(x).replace("'", "’"))


    for n in tqdm(range(len(data))):
        k1 = list(data.loc[n].fillna("NoValue").index)
        k1 = [str(a) for a in k1]
        k1 = ", ".join(k1)


        k = data.loc[n].fillna("NoValue").to_list()
        for i in range(len(k)):
            if str(type(k[i])) == "<class 'str'>":
                k[i] = "'" + k[i] + "'"
        k = [str(a) for a in k]
        k = [a.replace(",", "") for a in k]
        k = ", ".join(k)


        sql = '''INSERT INTO public.'''+ table +'''('''+k1+ ") VALUES ("+k+ ") ON CONFLICT DO NOTHING"
        try:
            engine.execute(sql)
        except:
            print("Fila " + str(n) +" no insertada")
            
    engine.dispose()

In [11]:
data1 = pd.read_csv("table_2.csv")


In [12]:
InsertInTable(data1,"tabla", database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432')

 52%|████████████████████████████████████████▏                                    | 1631/3129 [00:03<00:03, 418.29it/s]

Fila 1584 no insertada


 77%|███████████████████████████████████████████████████████████                  | 2399/3129 [00:05<00:02, 276.99it/s]

Fila 2358 no insertada
Fila 2368 no insertada
Fila 2372 no insertada
Fila 2378 no insertada
Fila 2379 no insertada
Fila 2381 no insertada
Fila 2387 no insertada
Fila 2396 no insertada


 79%|████████████████████████████████████████████████████████████▋                | 2465/3129 [00:06<00:02, 298.96it/s]

Fila 2431 no insertada


 91%|██████████████████████████████████████████████████████████████████████       | 2846/3129 [00:07<00:00, 395.64it/s]

Fila 2794 no insertada


100%|█████████████████████████████████████████████████████████████████████████████| 3129/3129 [00:07<00:00, 408.06it/s]


# fifth part (extract information from views)

In [13]:
def ReadView(sqlQuery, database, user, password, host, port):
    
    '''
   function that read a view
   
    database str : database name
    user str : user name
    password str : paswowrd of database
    host str: host of database
    port str: number of port
    sqlQuery str: Query to be execute 
    :return: null
   '''
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))

    
    try:
        data = pd.read_sql_query(sqlQuery,con=engine)
    except:
        print("could not be read view")

    engine.dispose()
    return(data)

In [14]:
query = "SELECT * FROM public.vista1"
view1 = ReadView(sqlQuery = query, database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432')
view1.head()

,keyword,id,date,avg_positive,avg_neutral,avg_negative
0,Tesla,8a09c213-c217-4c4e-98d5-4979bec2ba14,1.627747e+09,0.022,0.577,0.401
1,Biden,c913dbcc-cda8-4039-b355-ee3334e9b9e5,1.627673e+09,0.000,1.000,0.000
2,Biden,ed39a761-f819-44c6-9eba-b3e1f0be7480,1.627673e+09,0.042,0.708,0.250
3,Biden,04bc808b-5d54-4208-9909-60c547d4805e,1.627653e+09,0.000,1.000,0.000
4,Amazon,110020fa-86bd-4d4a-8b44-bb78e67622fd,1.627729e+09,0.134,0.866,0.000


In [15]:
query = "SELECT * FROM public.vista2"
view2 = ReadView(sqlQuery = query, database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432')
view2.head()

,date,keyword,count_positive,count_negative,count_neutral
0,1.627573e+09,Biden,0,0,1
1,1.627653e+09,Biden,0,0,2
2,1.627644e+09,Biden,0,0,1
3,1.627597e+09,Biden,0,0,4
4,1.627572e+09,Biden,0,0,1


# Sixth part (graphic)

In [16]:
def Graf(view1, view2):
    
    '''
   function that create some images
   
    view1 pandas.DataFrame:: data from the first view
    view2 pandas.DataFrame:: data from the second view
    :return: null
   '''
    

    view1.date = view1.date.apply(lambda x: datetime.fromtimestamp(x))
    view2.date = view2.date.apply(lambda x: datetime.fromtimestamp(x))

    data = view1[view1["date"]>="2021-07-28"][["date", 'avg_positive', "avg_neutral", "avg_negative"]]
    data = data.resample('60min', on='date').mean().reset_index()

    fig = px.bar(data, x='date', y=["avg_positive", "avg_neutral","avg_negative"],color_discrete_sequence = ["blue","gray","red"])
    fig.write_image("avarage_senti.png")


    data = view2[view2["date"]>="2021-07-28"][["date", 'count_positive', "count_neutral", "count_negative"]]
    df = data.resample('60min', on='date').sum()

    fig = px.line(df)
    fig.write_image("count1.png")

    df = data.resample('60min', on='date').sum()
    df1 = df.drop("count_neutral", axis = 1)

    fig = px.line(df1)
    fig.write_image("count2.png")

In [17]:
Graf(view1, view2)